In [1]:
country = 'Germany'
city = 'Berlin'

In [ ]:

import pandas as pd
import selenium

from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support import expected_conditions as EC

from housing_functions import get_driver

import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pprint
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import demjson3
import os
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import os
from dotenv import load_dotenv



#from housing_functions import random_click, get_driver, search_for_place

import json

import logging
import colorlog

import sys 
import zipfile


In [ ]:

import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pprint
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support import expected_conditions as EC

import os


def get_driver():
    driver = webdriver.Chrome()
    return driver


def random_click(driver):
    driver.execute_script("document.elementFromPoint(10, 10).click();")
    
    
    
def search_for_place(driver, url = 'https://housinganywhere.com/'):
    
    driver.get(url)

    
    place_to_search = driver.find_element(By.CSS_SELECTOR, ".css-19wcaby-input-input")
    place_to_search.send_keys('Berlin')
    
    
    search_button = driver.find_element(By.CSS_SELECTOR, "button[data-test-locator='Search and book']")
    search_button.click()
 
 
    random_click(driver)
    search_button.click()

def accept_cookies(driver):
    cookies_button = driver.find_element(By.ID,'onetrust-accept-btn-handler')
    cookies_button.click()
    
    
def scroll_page(driver):
    new_height = driver.execute_script("return document.body.scrollHeight")




def get_page_listings(driver):
    pass
    
def get_logger():
    handler = colorlog.StreamHandler()
    formatter = colorlog.ColoredFormatter(
        "%(log_color)s%(asctime)s [%(levelname)s] %(message)s",
        log_colors={
            "DEBUG": "cyan",
            "INFO": "green",
            "WARNING": "yellow",
            "ERROR": "red",
            "CRITICAL": "bold_red",
        }
    )
    handler.setFormatter(formatter)
    
    logger = colorlog.getLogger(__name__)
    logger.addHandler(logging.FileHandler("logging.txt", encoding="utf-8"))
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)
    return logger
    
def accept_cookies(driver):
    cookies_button = driver.find_element(By.ID,'onetrust-accept-btn-handler')
    cookies_button.click()
    
    
def get_container_listings(container):
    container.find_elements(By.CLASS_NAME,'css-1efwqj7-cardLink')
    return container


def create_directory_for_photos():
    if not os.path.exists(f'{country}_{city}_house_photos'):
        os.mkdir(f'{country}_{city}_house_photos')



def get_geo_data(driver):
    try:
        latitude, longitude, number_of_rooms = None, None, None
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        script_elements = soup.find_all('script', type='application/ld+json')
        for script in script_elements:
            if script.string:
                data = json.loads(script.string)
                
                if data.get('@type') != 'Accommodation':
                    continue
                geo_data = data.get('geo')
                latitude = geo_data.get('latitude')
                longitude = geo_data.get('longitude')
                number_of_rooms = data.get('numberOfRooms')         
    except :
        print('Inside the geo_data function, an Exception has occured')    
        return None, None, None
    
    return latitude, longitude, number_of_rooms
    


In [ ]:
def get_container(driver):
    page_container = driver.find_element(By.CLASS_NAME, 'css-wp5dsn-container')
    return page_container

def get_container_listings(container):
     rows = container.find_elements(By.CLASS_NAME,'css-1efwqj7-cardLink')
     return rows


def scroll_page(driver):
    new_height = driver.execute_script("return document.body.scrollHeight")


def scroll_down_and_wait_2_secs(driver):
    logger.info('In the scroll down and wait 2 secs function')
    driver.execute_script("window.scrollBy(0, 2000);")
    time.sleep(2)


# def scrape_half_page(driver):
    
#     listings_data_of_this_page = []
    
#     container = get_container(driver)
#     container_listings = get_container_listings(container)
    
    
#     for listing in container_listings:
        
#         listing_data =scrape_listing(listing,driver)
#         listings_data_of_this_page.append(listing_data)
    
    
#     return listings_data_of_this_page
    
    
def get_listing_tags(driver):
    try:
        tags_container = driver.find_element(By.CLASS_NAME, 'css-q6fy6c-highlightContainer')
        if tags_container:
            container_elements = tags_container.find_elements(By.CLASS_NAME, 'css-1e5azn1-highlightItem')
            tags = []
            for tag_element in container_elements:
                tag = tag_element.text
                tags.append(tag)
            return tags
    except:
        return [None]
        
def get_images(driver):
    logger.info('Inside the get_images function')
    more_photos_button = driver.find_element(By.CLASS_NAME,'css-13emeri-tile-tileButton')
    more_photos_button.click()
    images = driver.find_elements(By.TAG_NAME, 'img')   
    return images
        
def scrape_listing_photos_and_create_their_file(driver, listing_id:str):
    logger.info('Inside the scrape_listing_photos_and_create_their_file function ')
    
    logger.info(f'Trying to create the {country}_{city}_house_photos directory')
    folder = os.path.join(f'{country}_{city}_house_photos', listing_id)
    os.makedirs(folder, exist_ok=True)

        
    images = get_images(driver)
    logger.info('Got more images')
    
    unique_images = set(images)
    
    zip_file_path = os.path.join(folder, f"{listing_id}_photos.zip")
    
    with zipfile.ZipFile(zip_file_path, 'w', compression=zipfile.ZIP_DEFLATED) as f1:
        logger.info(f'Created the zip path {zip_file_path}')    
        
        for i, image in enumerate(unique_images):
            logger.info(f'Enumerating the images of listing with id; {listing_id}')
            
            size = image.size
            height = size['height']
            if height >= 45:
                image_url = image.get_attribute('src') or image.get_attribute('data-src') or image.get_attribute('data-lazy') or image.get_attribute('data-original')
                
                if image_url.startswith("http"):

                    response = requests.get(image_url)
                    image_to_be_written = response.content
                    
                    image_bytes = len(image_to_be_written)
                    
                    if image_bytes > 100 * 1000:

                        image_name = listing_id + '_' + str(i) + '.jpg'
                        f1.writestr(image_name, image_to_be_written)
                        logger.info(f'Successfully wrote image, with image name {image_name}')
    
                else:
                    logger.error(f'image {i} doesnt start with http')
                    logger.error(image_url)

    
    
def create_metadata_file_in_the_listings_folder(listing_id,title,price,area,description,full_description,tags,latitude,longitude,number_of_rooms):
    try:
        print(listing_id.__class__)
        if not isinstance(listing_id,str):
            logger.error(f'Listing with id; {listing_id} is not of type; str')
            raise ValueError
        folder = os.path.join(f'{country}_{city}_house_photos', listing_id)    
        os.makedirs(folder, exist_ok=True)
        logger.info(f'Created folder or folder already exists for listing with id; {listing_id}')
        file = os.path.join(folder,'.metadata')
        with open (file, 'w', encoding='utf-8') as f1:
            meta_data = {
                'listing_id' : listing_id,
                'title' : title,
                'price' : price,
                'area' : area,
                'description' : description,
                'full_description' : full_description,
                'latitude' : latitude,
                'longitude'  : longitude,
                'number_of_rooms' : number_of_rooms
            }
            for i, tag in enumerate(tags):
                key = f'tag_{i}'
                meta_data[key] = tag
                
            json_meta_data = json.dumps(meta_data, ensure_ascii= False, indent=4)
            f1.write(json_meta_data)
            logger.info(f'Wrote meta_data file for listing with id; {listing_id}')
    except Exception :
        logger.error(f'Encountered error during the writing of the Meta Data file for listing with id; {listing_id}')
        driver.close()
        
        
        
def go_up_a_bit(driver):
    logger.info('******************* In the go up a bit function ******************')
    driver.execute_script("window.scrollBy(0, -2000);")   
        
        
def get_listing_text_attributes(driver):
    try:
        title = driver.find_element(By.CLASS_NAME,'css-1ql5bbl').text
        price = driver.find_element(By.CLASS_NAME, 'css-1bop1zx-pricingContent').text
        area = driver.find_element(By.CLASS_NAME, 'css-2ccjfp').text
        description = driver.find_element(By.CLASS_NAME, 'css-31lj5q').text
        full_description = driver.find_element(By.CLASS_NAME, 'css-1liw7jd-preWrap-breakWord').text

        return title, price, area, description, full_description
    
    except Exception:
        return (None,None,None,None,None,)
        


def get_listing_id(driver):
    current_url = driver.current_url
    if '/ut' in current_url:
        listing_id = current_url.split('/ut')[1].split('/de')[0]
    else:    
        listing_id = current_url.split('-')[-1]
    return listing_id



def scrape_listing(listing, driver, original_window):
    try:
        listing.click()
    except Exception:
        logger.error('Didn t manage to sclick on listing.')
    
    try:
        driver.switch_to.window(driver.window_handles[-1])
        listing_id = get_listing_id(driver)
        logger.info(f'Acquired listing id {listing_id}')
        
        title, price, area, description, full_description = get_listing_text_attributes(driver)
        logger.info(f'Title : {title}, price : {price}, area : {area}, description : {description}, fulldescription : {full_description}')
        
        tags = get_listing_tags(driver)
        logger.info(f'Tags : {tags}' )
        
        scrape_listing_photos_and_create_their_file(driver,listing_id)
        logger.info('Scraped photos')
        
        latitude, longitude, number_of_rooms = get_geo_data(driver)
        logger.info(f'Latitude {latitude}, longitude : {longitude}, Number of Rooms : {number_of_rooms}')

        create_metadata_file_in_the_listings_folder(listing_id,title,price,area,description,full_description,tags,latitude,longitude,number_of_rooms)
        logger.info(f'Created metadata file for listing with id ; {listing_id}')
    
    except Exception:
        logger.error('something failed when scraping the data')
    finally:
        
        if driver.current_window_handle != original_window:
            driver.close()
            driver.switch_to.window(original_window)


<button class="MuiButtonBase-root MuiButton-root MuiButton-contained MuiButton-containedPrimary MuiButton-sizeMedium MuiButton-containedSizeMedium MuiButton-fullWidth MuiButton-root MuiButton-contained MuiButton-containedPrimary MuiButton-sizeMedium MuiButton-containedSizeMedium MuiButton-fullWidth css-tkyhxc-button-button" tabindex="0" type="submit" data-test-locator="Search and book">Search<span class="MuiTouchRipple-root css-w0pj6f"></span></button>

******** PROGRAM STARTS

In [5]:
logger = get_logger()
driver = get_driver()
create_directory_for_photos()
#ipython = get_ipython()


In [6]:
tries = 0
while True:
    try:
        tries += 1
        if tries == 2:
            os.execv(sys.executable, [sys.executable] + sys.argv)

        search_for_place(driver)
        accept_cookies(driver)
        break

    except Exception :
        logger.error('Exception while trying to search for place')

In [7]:
def scrape_half_page(driver):
    try:
        logger.info('Trying to get the container element')
        container = driver.find_element(By.CLASS_NAME, 'css-wp5dsn-container')
        
        scroll_page(driver)
        logger.info('Trying to get the container\'s listings')
        container_listings = get_container_listings(container)
        
        original_window = driver.current_window_handle

        logger.info('Iterating over the container\'s listings')
        for  listing in container_listings:
            try:
                logger.info('Trying to scrape a listing from the container')
                listing_data = scrape_listing(listing, driver, original_window)
            except ElementClickInterceptedException:
                logger.error('Failed to scrape a listing from the container')
    except Exception :
        logger.error('Undefined error')

    finally:
        print('containers;', len(container_listings))

def scrape_page(driver):
    '''
    For each page, scrape its upper half first.
    Then scroll down a bit, to get more container listings, and scrape the other half
    '''
    scrape_half_page(driver)
    driver.execute_script("window.scrollBy(0, 2000);")
    scrape_half_page(driver)


def go_to_next_page(driver):
    logger.info('Trying to go to the next Page')
    next_button = driver.find_element(By.CSS_SELECTOR, "button[aria-label='Go to next page']")
    next_button.click()
    logger.info('Clicked on next Page')


def scrape_pages(driver):
    logger.info('Started Scraping Pages')
    pages_scraped = 0
    try:
        while True:
            try:
                logger.info(f'Trying to Scrape the {pages_scraped}th Page')
                scrape_page(driver)
                logger.info(f'Successfuly scraped the {pages_scraped}th Page')
                
                pages_scraped += 1        
                
                logger.info(f'Trying to click on the {pages_scraped}th Page')
                
                scroll_down_and_wait_2_secs(driver)

                go_to_next_page(driver)
                
                if pages_scraped == 100: # we do not need to scrape more than 100 pages
                    logger.info('Scraped 100 pages, Now exiting')
                    break
            except Exception as e:
                logger.error(f'Encounterd Error during the scraping of the {pages_scraped}th Page', exc_info=True)
                logger.error('Attempting to go up a bit')
                go_up_a_bit(driver)
    finally:
        driver.close()

In [8]:
scrape_pages(driver)

2025-09-28 17:30:07,128 [INFO] Started Scraping Pages
2025-09-28 17:30:07,130 [INFO] Trying to Scrape the 0th Page
2025-09-28 17:30:07,131 [INFO] Trying to get the container element
2025-09-28 17:30:07,142 [INFO] Trying to get the container's listings
2025-09-28 17:30:07,153 [INFO] Iterating over the container's listings
2025-09-28 17:30:07,154 [INFO] Trying to scrape a listing from the container
2025-09-28 17:30:09,363 [INFO] Acquired listing id 2342470
2025-09-28 17:30:09,440 [INFO] Title : Studyo, price : From
€850 /month, area : 16 m²+, description : Cleaning in common areas, fulldescription : BOOK NOW & get the December rent for free!*

Find your preferred room, book online before 30th September and grab one month rent free
in December.
Spend your December budget on gifts and fun instead of rent.

With us you’ll find more than just an apartment. You’ll find a place that feels like home from day one, where you can focus on your studies, build lasting
friendships, and become part of

<class 'str'>


2025-09-28 17:30:20,215 [INFO] Acquired listing id 1148761
2025-09-28 17:30:20,324 [INFO] Title : None, price : None, area : None, description : None, fulldescription : None
2025-09-28 17:30:20,334 [INFO] Tags : [None]
2025-09-28 17:30:20,334 [INFO] Inside the scrape_listing_photos_and_create_their_file function 
2025-09-28 17:30:20,335 [INFO] Trying to create the Germany_Berlin_house_photos directory
2025-09-28 17:30:20,336 [INFO] Inside the get_images function
2025-09-28 17:30:20,543 [INFO] Got more images
2025-09-28 17:30:20,549 [INFO] Created the zip path Germany_Berlin_house_photos\1148761\1148761_photos.zip
2025-09-28 17:30:20,550 [INFO] Enumerating the images of listing with id; 1148761
2025-09-28 17:30:21,127 [INFO] Successfully wrote image, with image name 1148761_0.jpg
2025-09-28 17:30:21,128 [INFO] Enumerating the images of listing with id; 1148761
2025-09-28 17:30:21,457 [INFO] Successfully wrote image, with image name 1148761_1.jpg
2025-09-28 17:30:21,458 [INFO] Enumeratin

<class 'str'>


2025-09-28 17:30:42,153 [INFO] Acquired listing id 1145984
2025-09-28 17:30:42,234 [INFO] Title : Neonwood Adlershof, price : From
€765 /month, area : 17 m²+, description : Cleaning in common areas, fulldescription : Life at Berlin Adlershof
295 apartments, 295+ students... be one of us & meet extraordinary people. Adlershof is not only a home to renowned non-university research institutions, six institutes of the Humboldt University and around 1,200 companies that are perfect for internships, but also – and this is the best part – our brand new building. Be a part of our community and meet exciting people like you. Our student residence is just a stone’s throw away from the Humboldt University of Berlin – C...
2025-09-28 17:30:42,244 [INFO] Tags : [None]
2025-09-28 17:30:42,244 [INFO] Inside the scrape_listing_photos_and_create_their_file function 
2025-09-28 17:30:42,245 [INFO] Trying to create the Germany_Berlin_house_photos directory
2025-09-28 17:30:42,246 [INFO] Inside the get_im

...

2025-09-28 17:44:47,734 [ERROR] Didn t manage to sclick on listing.
2025-09-28 17:44:47,739 [INFO] Acquired listing id Germany?page=3
2025-09-28 17:44:47,744 [INFO] Title : None, price : None, area : None, description : None, fulldescription : None
2025-09-28 17:44:47,750 [INFO] Tags : [None]
2025-09-28 17:44:47,751 [INFO] Inside the scrape_listing_photos_and_create_their_file function 
2025-09-28 17:44:47,751 [INFO] Trying to create the Germany_Berlin_house_photos directory
2025-09-28 17:44:47,752 [ERROR] something failed when scraping the data
2025-09-28 17:44:47,754 [INFO] Trying to scrape a listing from the container
2025-09-28 17:44:48,830 [ERROR] Didn t manage to sclick on listing.
2025-09-28 17:44:48,837 [INFO] Acquired listing id Germany?page=3
2025-09-28 17:44:48,842 [INFO] Title : None, price : None, area : None, description : None, fulldescription : None
2025-09-28 17:44:48,849 [INFO] Tags : [None]
2025-09-28 17:44:48,850 [INFO] Inside the scrape_listing_photos_and_create_th

containers; 20


2025-09-28 17:45:10,658 [INFO] Trying to go to the next Page
2025-09-28 17:45:11,737 [ERROR] Encounterd Error during the scraping of the 14th Page
Traceback (most recent call last):
  File "C:\Users\steve\AppData\Local\Temp\ipykernel_19320\1086546137.py", line 58, in scrape_pages
    go_to_next_page(driver)
  File "C:\Users\steve\AppData\Local\Temp\ipykernel_19320\1086546137.py", line 38, in go_to_next_page
    next_button.click()
  File "c:\Users\steve\web-scraping\.venv\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 119, in click
    self._execute(Command.CLICK_ELEMENT)
  File "c:\Users\steve\web-scraping\.venv\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 572, in _execute
    return self._parent.execute(command, params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\steve\web-scraping\.venv\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 429, in execute
    self.error_handler.check_response(response)
  File "c:\

containers; 20


KeyboardInterrupt: 

Next feature testing

In [ ]:

# def go_to_next_page(driver):
#     logger.info('Trying to go to the next Page')
#     next_button = driver.find_element(By.CSS_SELECTOR, "button[aria-label='Go to next page']")
#     next_button.click()
#     logger.info('Clicked on next Page')


In [ ]:
# logger = get_logger()
# driver = get_driver()
# driver.get('https://housinganywhere.com')
# search_for_place(driver)

In [ ]:
# driver.execute_script("window.scrollBy(0, 4000);")
# time.sleep(2)
# go_to_next_page(driver)


In [ ]:
#driver.execute_script("window.scrollTo(200, document.body.scrollHeight);")

#go_to_next_page(driver)


In [ ]:
go_to_next_page(driver)